In [5]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
import sklearn
import xgboost as xgb
import re
import time
from BusinessHours import BusinessHours as bh
from pandas.tseries.holiday import USFederalHolidayCalendar as calendar

In [6]:
df = pd.read_csv('train_aWnotuB.csv')
df_test = pd.read_csv('test_BdBKkAj.csv')

In [7]:
df.head()

#df.describe()

,DateTime,Junction,Vehicles,ID
0,2015-11-01 00:00:00,1,15,20151101001
1,2015-11-01 01:00:00,1,13,20151101011
2,2015-11-01 02:00:00,1,10,20151101021
3,2015-11-01 03:00:00,1,7,20151101031
4,2015-11-01 04:00:00,1,9,20151101041


In [46]:
#df = df_train[df.columns.difference(['ID'])]

In [9]:
df['date'] = pd.to_datetime(df['DateTime'])
dti = pd.DatetimeIndex(df['date'])
holidays = calendar().holidays(start=dti.min(), end=dti.max())

df['year'] = dti.year
df['quarter'] = dti.quarter
df['month'] = dti.month
df['week'] = dti.week
df['weekofyear'] = dti.weekofyear
df['day'] = dti.day
df['dayofweek'] = dti.dayofweek
df['dayofyear'] = dti.dayofyear
df['hour'] = dti.hour
df['minofday'] = dti.hour*60+dti.minute
df['bhour'] = df['hour']
df['bhour'] = df['bhour'].replace(range(1,8), 0).replace(range(8,19), 1).replace(range(19,24), 0)
df['holiday'] = dti.isin(holidays)
df['holiday'] = df['holiday'].map({False:0, True:1})

# df['year'], df['month'], df['week'], df['day'], df['dayofweek'], df['hour'], df['minofday'] = \
# df['date'].dt.year, df['date'].dt.month, df['date'].dt.week, df['date'].dt.day, df['date'].dt.dayofweek, df['date'].dt.hour, df['date'].dt.hour*60+df['date'].dt.minute

In [129]:
df.date = df.DateTime.apply(lambda x: time.mktime(pd.Timestamp(x).to_datetime().timetuple()))

/home/smrutiranjans/miniconda3/envs/ai/lib/python3.6/site-packages/pandas/core/series.py:2355: FutureWarning: to_datetime is deprecated. Use self.to_pydatetime()
  mapped = lib.map_infer(values, f, convert=convert_dtype)


In [11]:
from tsfresh import extract_relevant_features
from tsfresh import extract_features
extracted_features = extract_relevant_features(df, df.Vehicles.values, column_sort="DateTime", column_id="ID", column_kind="Junction", column_value="Vehicles")

Feature Extraction: 100%|██████████| 10/10 [24:41<00:00, 148.19s/it]
 '1__agg_linear_trend__f_agg_"max"__chunk_len_10__attr_"rvalue"'
 '1__agg_linear_trend__f_agg_"max"__chunk_len_10__attr_"slope"' ...,
 '4__spkt_welch_density__coeff_2' '4__spkt_welch_density__coeff_5'
 '4__spkt_welch_density__coeff_8'] did not have any finite values. Filling with zeros.


In [14]:
extracted_features.columns

Index(['2__cwt_coefficients__widths_(2, 5, 10, 20)__coeff_0__w_5',
       '2__cwt_coefficients__widths_(2, 5, 10, 20)__coeff_0__w_2',
       '2__cwt_coefficients__widths_(2, 5, 10, 20)__coeff_0__w_20',
       '2__fft_coefficient__coeff_0__attr_"abs"',
       '2__fft_coefficient__coeff_0__attr_"real"', '2__maximum', '2__mean',
       '2__median', '2__minimum', '2__quantile__q_0.1', '2__quantile__q_0.2',
       '2__quantile__q_0.3', '2__quantile__q_0.4', '2__quantile__q_0.6',
       '2__quantile__q_0.7', '2__quantile__q_0.8', '2__quantile__q_0.9',
       '2__sum_values',
       '2__cwt_coefficients__widths_(2, 5, 10, 20)__coeff_0__w_10',
       '2__abs_energy', '1__abs_energy', '1__quantile__q_0.9',
       '1__cwt_coefficients__widths_(2, 5, 10, 20)__coeff_0__w_10',
       '1__cwt_coefficients__widths_(2, 5, 10, 20)__coeff_0__w_2',
       '1__cwt_coefficients__widths_(2, 5, 10, 20)__coeff_0__w_20',
       '1__cwt_coefficients__widths_(2, 5, 10, 20)__coeff_0__w_5',
       '1__sum_values',

In [21]:
temps = DataFrame(df.Vehicles)
# dataframe = concat([temps.shift(1), temps], axis=1)
# dataframe.columns = ['t-1', 't+1']
vehicles
dataframe = concat([temps.shift(3), temps.shift(2), temps.shift(1), temps], axis=1)
dataframe.columns = ['t-3', 't-2', 't-1', 't+1']
print(dataframe.head(5))

,DateTime,Junction,Vehicles,ID,date,year,month,week,day
0,2015-11-01 00:00:00,1,15,20151101001,2015-11-01 00:00:00,2015,11,44,1
1,2015-11-01 01:00:00,1,13,20151101011,2015-11-01 01:00:00,2015,11,44,1
2,2015-11-01 02:00:00,1,10,20151101021,2015-11-01 02:00:00,2015,11,44,1
3,2015-11-01 03:00:00,1,7,20151101031,2015-11-01 03:00:00,2015,11,44,1
4,2015-11-01 04:00:00,1,9,20151101041,2015-11-01 04:00:00,2015,11,44,1
5,2015-11-01 05:00:00,1,6,20151101051,2015-11-01 05:00:00,2015,11,44,1
6,2015-11-01 06:00:00,1,9,20151101061,2015-11-01 06:00:00,2015,11,44,1
7,2015-11-01 07:00:00,1,8,20151101071,2015-11-01 07:00:00,2015,11,44,1
8,2015-11-01 08:00:00,1,11,20151101081,2015-11-01 08:00:00,2015,11,44,1
9,2015-11-01 09:00:00,1,12,20151101091,2015-11-01 09:00:00,2015,11,44,1
